In [ ]:
import os


In [ ]:
cd drive/Shareddrives/KOHI_의료영상1팀/

/content/drive/Shareddrives/KOHI_의료영상1팀


## 최선 선생님 데이터 파일 압축 해제

In [ ]:
train_data_path = './Training_Data'

zip_list = os.listdir(train_data_path)

In [ ]:
zip_list

['test_image_c-20211023T054251Z-001.zip',
 'test_mask_dilate-20211023T054620Z-001.zip',
 'train_image_c-20211023T054651Z-001.zip',
 'train_mask_dilate-20211023T055145Z-001.zip',
 'test_mask_dilate-20211023T054620Z-001',
 'train_image_c-20211023T054651Z-001',
 'train_mask_dilate-20211023T055145Z-001']

In [ ]:
from zipfile import ZipFile
import os
import glob

In [ ]:
for zip in zip_list:
  if '.zip' not in zip:
    zip_list.remove(zip)

print(zip_list)

['test_image_c-20211023T054251Z-001.zip', 'test_mask_dilate-20211023T054620Z-001.zip', 'train_image_c-20211023T054651Z-001.zip', 'train_mask_dilate-20211023T055145Z-001.zip', 'train_image_c-20211023T054651Z-001']


In [ ]:
zip_list

['test_image_c-20211023T054251Z-001.zip',
 'test_mask_dilate-20211023T054620Z-001.zip',
 'train_image_c-20211023T054651Z-001.zip',
 'train_mask_dilate-20211023T055145Z-001.zip',
 'train_image_c-20211023T054651Z-001']

In [ ]:
image_index = 0

for zip in zip_list[:-1]:
  zip_name = (zip[:-4])

  try:
    _zip = ZipFile(os.path.join(train_data_path, zip))
    print(f'{os.path.join(train_data_path, zip)} Good!')
  except:
    print(os.path.join(train_data_path, zip))
  
  ckpt_dir=os.path.join(train_data_path, zip_name)
  
  if not os.path.isdir(ckpt_dir):
      os.mkdir(ckpt_dir)

  _zip.extractall(ckpt_dir)
  
  _zip.close()    

./Training_Data/test_image_c-20211023T054251Z-001.zip Good!
./Training_Data/test_mask_dilate-20211023T054620Z-001.zip Good!
./Training_Data/train_image_c-20211023T054651Z-001.zip Good!
./Training_Data/train_mask_dilate-20211023T055145Z-001.zip Good!


### deformation 데이터만 남기고 나머지 삭제

In [ ]:
train_data_path = './Training_Data'

zip_list = os.listdir(train_data_path)
zip_list

['test_image_c',
 'train_image_c',
 'test_mask_dilate',
 'train_mask_dilate',
 'test_image_c-20211023T054251Z-001.zip',
 'test_mask_dilate-20211023T054620Z-001.zip',
 'train_image_c-20211023T054651Z-001.zip',
 'train_mask_dilate-20211023T055145Z-001.zip']

In [ ]:
zip_list = os.listdir(train_data_path)

for zip in zip_list:
  if 'zip' in zip:
    print(zip)
    zip_list.remove(zip)

image_dirs = zip_list[:4]

In [ ]:
image_dirs

['test_image_c', 'train_image_c', 'test_mask_dilate', 'train_mask_dilate']

In [ ]:
from tqdm import tqdm

In [ ]:
for dir in image_dirs:
  image_list = os.listdir(os.path.join(train_data_path, dir))
  for image in tqdm(image_list):
    print(os.path.join(train_data_path, dir, image))
    if not 'trans' in image:
      print(os.path.join(train_data_path, dir, image))
      os.remove(os.path.join(train_data_path, dir, image))

# 데이터 병합

### Deforamtion Data 이동


In [ ]:
import shutil

In [ ]:
for dir in image_dirs:
  image_list = os.listdir(os.path.join(train_data_path, dir))
  if 'mask' in dir:
    for image in image_list:
      present_path = os.path.join(train_data_path, dir, image)
      copy_path = os.path.join(train_data_path, 'mask', image)
      shutil.copy(present_path, copy_path)
  elif 'image' in dir:
    for image in image_list:
      present_path = os.path.join(train_data_path, dir, image)
      copy_path = os.path.join(train_data_path, 'image', image)
      shutil.copy(present_path, copy_path)

### Check file 개수

In [ ]:
mask = 0
image = 0
for dir in image_dirs:
  image_list = os.listdir(os.path.join(train_data_path, dir))
  if 'mask' in dir:
    mask += len(image_list)
  elif 'image' in dir:
    image += len(image_list)

print(f'mask: {mask}, image: {image}')      

mask: 1407, image: 1407


In [ ]:
mask_dir = len(os.listdir(os.path.join(train_data_path, 'mask')))
image_dir = len(os.listdir(os.path.join(train_data_path, 'image')))
print(f'mask: {mask_dir}, image: {image_dir}')      

mask: 1407, image: 1407


# Dehazing 이미지, labeled mask 이동
(labeled mask의 경우 .tiff를 .png로 변환하여 저장)

In [ ]:
import pandas as pd

In [ ]:
data_type = 'dehazing'

In [ ]:
df = pd.read_csv(f'Data/labeled-final.csv')

scoliosis_label_dir = 'Data/labeled-scoliosis'
normal_label_dir = 'Data/labeled-normal'

input_dir = 'Data/labeled-final'
input_list = os.listdir(input_dir)

In [ ]:
train_data_path = './Training_Data'

## Dehazing 이미지 먼저 이동!

In [ ]:
dataset = []

for input in tqdm(input_list):
  try:
    label = df[df['ImageID']==input]['Labels'].values[0]
  except:
    print(f'{input} is not in csv file')
    pass

  present_path = os.path.join(input_dir, input)
  copy_path = os.path.join(train_data_path, '_image', input)
  shutil.copy(present_path, copy_path)

100%|██████████| 917/917 [06:42<00:00,  2.28it/s]


전체 image 데이터가 잘 이동됐는지 확인 후 image dir로 이동

In [ ]:
image_dir = len(os.listdir(os.path.join(train_data_path, '_image')))
print(f'image: {image_dir}')      

image: 917


In [ ]:
image_list = os.listdir(os.path.join(train_data_path, '_image'))

for image in tqdm(image_list):
  present_path = os.path.join(train_data_path, '_image', image)
  copy_path = os.path.join(train_data_path, 'image', image)
  shutil.copy(present_path, copy_path)

100%|██████████| 917/917 [00:49<00:00, 18.43it/s]


## label mask 형식 변환

In [ ]:
len(os.listdir(os.path.join(normal_label_dir)))

496

In [ ]:
len(os.listdir(os.path.join(scoliosis_label_dir)))

427

In [ ]:
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np

In [ ]:
normal = 0
scoliosis = 0

def label_preprocessing(label):
  label = cv.bitwise_not(label)   
  kernel = np.ones((3, 3), np.uint8)
  label = cv.dilate(label, kernel, iterations=80)
  return label


for input in tqdm(input_list):
  try:
    label = df[df['ImageID']==input]['Labels'].values[0]
  except:
    print(f'{input} is not in csv file')
    pass
    
  if 'normal' in label:
    try:
      tiff = glob.glob(os.path.join(normal_label_dir, input[:-4], "*.tiff"))[0] 
    except:
      print(f'{input[:-4]} - no tiff file!')
      exit(-1)
  elif 'scoliosis' in label:
    try:
      tiff = glob.glob(os.path.join(scoliosis_label_dir, input[:-4], "*.tiff"))[0]
    except:
      print(f'{input[:-4]} - no tiff file!')
      exit(-1)
  
  label = cv.imread(tiff)
  label = cv.cvtColor(label, cv.COLOR_BGR2GRAY)  
  label = label_preprocessing(label)

  copy_path = os.path.join(train_data_path, '_mask', input)

  cv.imwrite(copy_path, label)

  # # plt.imshow(label, cmap='gray')
  # break


100%|██████████| 917/917 [17:50<00:00,  1.17s/it]


In [ ]:
mask_dir = len(os.listdir(os.path.join(train_data_path, '_mask')))
print(f'mask: {mask_dir}')      

mask: 917


In [ ]:
mask_list = os.listdir(os.path.join(train_data_path, '_mask'))

for mask in tqdm(mask_list):
  present_path = os.path.join(train_data_path, '_mask', mask)
  copy_path = os.path.join(train_data_path, 'mask', mask)
  shutil.copy(present_path, copy_path)

100%|██████████| 917/917 [00:09<00:00, 100.80it/s]


# 전체 이미지 수 확인

In [ ]:
mask_dir = len(os.listdir(os.path.join(train_data_path, 'mask')))
image_dir = len(os.listdir(os.path.join(train_data_path, 'image')))
print(f'mask: {mask_dir}, image: {image_dir}')      

mask: 2324, image: 2324


# 이름 변경

In [ ]:
image_dir = 'Training_Data/image'
image_list = os.listdir('Training_Data/image')

In [ ]:
for image in image_list:
  if 'trans' in image:
    file_oldname = os.path.join(image_dir, image)
    file_newname = os.path.join(image_dir, image[2:])
        
    os.rename(file_oldname, file_newname)
    print(f'{file_oldname} to {file_newname}')
  #   file_newname_newfile = os.path.join("c:\\Folder-1", "NewFileName.NewExtension")

    

In [ ]:
print(len(os.listdir('Training_Data/image')))
print(len(os.listdir('Training_Data/mask')))

2323
2324


In [ ]:
amsk =os.listdir('Training_Data/mask')
image = os.listdir('Training_Data/image')

temp3 = list(set(amsk) - set(image)) 

In [ ]:
temp3

[]

# ★ line image 이동

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
import cv2 as cv
import matplotlib.pyplot as plt

In [ ]:
cd drive/Shareddrives/KOHI_의료영상1팀/

[Errno 2] No such file or directory: 'drive/Shareddrives/KOHI_의료영상1팀/'
/content/drive/Shareddrives/KOHI_의료영상1팀


In [ ]:
scoliosis_label_dir = 'Data/labeled-scoliosis'
normal_label_dir = 'Data/labeled-normal'

save_normal_label = 'Data/labeled-final'

label_dirs = [scoliosis_label_dir, normal_label_dir]
print(label_dirs)

['Data/labeled-scoliosis', 'Data/labeled-normal']


In [ ]:
dataset = []


for label_dir in label_dirs:
  dirs = os.listdir(label_dir)
  print(f'Get {label_dir} files...')
  for dir in tqdm(dirs):
    present_dir = os.path.join(label_dir, dir)
    try:
      png = glob.glob(os.path.join(present_dir, "*.png"))[0]
    except:
      print(f'{dir} - no png file!')
      continue
    
    try:
      tiff = glob.glob(os.path.join(present_dir, "*.tiff"))[0]
    except:
      print(f'{dir} - no tiff file!')
      continue

    dataset.append({'name':dir, 'image_path':png, 'label_path':tiff})

Get Data/labeled-scoliosis files...


100%|██████████| 427/427 [00:28<00:00, 14.76it/s] 


Get Data/labeled-normal files...


100%|██████████| 496/496 [00:29<00:00, 16.83it/s] 

216840111366964013575787468332013010132737229_02-012-057 (1) - no png file!
216840111366964013575787468332013010111727545_02-012-005 (1) - no png file!
216840111366964013829543166512013357084835266_02-094-007 (1) - no png file!
216840111366964013575787468332013015092215559_02-010-177 (1) - no png file!
216840111366964013575787468332013017121350696_02-011-172 (1) - no png file!
216840111366964013575787468332013017110957549_02-011-121 (1) - no png file!


In [ ]:
len(dataset)

917

In [ ]:
output_name_list = os.listdir('./Training_Data/output')

In [ ]:
len(output_name_list)

232

In [ ]:
normal = 0
scoliosis = 0
count=0

for input in tqdm(output_name_list):
  input = input + '.png'
  try:
    label = df[df['ImageID']==input]['Labels'].values[0]
  except:
    print(f'{input} is not in csv file')
    pass
    
  if 'normal' in label:
    try:
      tiff = glob.glob(os.path.join(normal_label_dir, input[:-4], "*.tiff"))[0] 
    except:
      print(f'{input[:-4]} - no tiff file!')
      exit(-1)
  elif 'scoliosis' in label:
    try:
      tiff = glob.glob(os.path.join(scoliosis_label_dir, input[:-4], "*.tiff"))[0]
    except:
      print(f'{input[:-4]} - no tiff file!')
      exit(-1)
  
  label = cv.imread(tiff)
  label = cv.cvtColor(label, cv.COLOR_BGR2GRAY)  
  label = cv.bitwise_not(label)   

  copy_path = os.path.join('./Training_Data', 'line', input)

  cv.imwrite(copy_path, label)
  # # plt.imshow(label, cmap='gray')


100%|██████████| 232/232 [00:13<00:00, 17.76it/s]


In [32]:
len(os.listdir(os.path.join('./Training_Data', '_output')))

232